## Aula 13 - Trabalhando com Schedulers no OpenMP

### Scheduler

#### Tempos obtidos:

default took 0.000205924 seconds.

schedule(static) took 3.0743e-06 seconds.

schedule(dynamic) took 6.89551e-06 seconds.

schedule(guided) took 4.73764e-06 seconds.

schedule(auto) took 3.28384e-06 seconds.

default took 2.74461e-06 seconds.

schedule(static) took 2.69711e-06 seconds.

schedule(dynamic) took 5.0785e-06 seconds.

schedule(guided) took 3.93763e-06 seconds.

schedule(auto) took 2.8545e-06 seconds.

default took 2.82377e-06 seconds.

schedule(static) took 2.63844e-06 seconds.

schedule(dynamic) took 8.14162e-06 seconds.

schedule(guided) took 3.49712e-06 seconds.

schedule(auto) took 2.81632e-06 seconds.


Average execution times (3 runs):

default:               7.04974e-05 seconds.

schedule(static):      2.80328e-06 seconds.

schedule(dynamic):     6.70521e-06 seconds.

schedule(guided):      4.05746e-06 seconds.

schedule(auto):        2.98489e-06 seconds.



### 1. Qual *scheduler* apresentou o menor tempo médio?
- **`schedule(static)`** teve o menor tempo médio, com aproximadamente `2.80328e-06` segundos. Logo após, `schedule(auto)` também apresentou um desempenho próximo, com `2.98489e-06` segundos.

### 2. Algum *scheduler* teve variações significativas entre as execuções? Se sim, por quê?
- **`default`** apresentou a maior variação entre as execuções. O primeiro valor (`0.000205924` segundos) foi significativamente maior do que os dois subsequentes.
- Essa variação pode ocorrer porque o *scheduler* padrão do OpenMP (default) depende da implementação específica do compilador e pode não otimizar a distribuição de trabalho de forma tão eficiente quanto `static`, `dynamic`, ou `guided` em certas situações.
- Além disso, dependendo da carga de trabalho e da inicialização das threads, a primeira execução pode enfrentar mais sobrecarga, enquanto as subsequentes beneficiam de um aquecimento das threads.

### 3. Alguma característica específica do trabalho (como carga de dados, balanceamento) parece ter influenciado o comportamento de um *scheduler* em particular?
- A carga de trabalho é relativamente pequena (`n = 64`), o que favorece agendamentos que têm baixa sobrecarga na distribuição das tarefas, como `static` e `auto`.
- **`static`** tende a ser mais eficiente em situações em que a carga de trabalho é uniforme e previsível, porque divide o trabalho em blocos iguais entre as threads.
- **`dynamic`** pode apresentar um pouco mais de sobrecarga em situações de carga pequena devido à necessidade de redistribuir dinamicamente os blocos de trabalho. Isso explica por que ele é um pouco mais lento do que `static` nos seus tempos.
- **`guided`** também mostrou um desempenho bom, mas um pouco mais lento do que `static`, pois ajusta o tamanho dos blocos conforme a execução, reduzindo a granularidade ao longo do tempo.
- **`auto`** permite que o compilador decida a melhor estratégia, e ele se comportou bem aqui, possivelmente adotando uma estratégia similar a `static` devido à carga leve e previsível.

### Conclusão:
- Para cargas pequenas como `n = 64`, o agendamento `static` é particularmente eficiente, pois minimiza a sobrecarga de gerenciamento de tarefas. 
- Em cenários com cargas de trabalho maiores ou com variabilidade significativa, `dynamic` ou `guided` podem ser mais vantajosos, pois permitem uma melhor adaptação ao balanceamento das tarefas.
- A diferença significativa entre o primeiro tempo de `default` e os demais mostra como a inicialização e a forma como a carga de trabalho é tratada podem impactar os tempos.

### Calculo do PI

#### Parallel for

MIN_BLK = $1024\cdot1024\cdot256$

For 2147483648 steps, pi = 3.14159265358979 in 1.07848663907498 secs

---

MIN_BLK = $1024\cdot1024\cdot64$

For 2147483648 steps, pi = 3.14159265358979 in 1.08539796154946 secs

---

MIN_BLK = $1024\cdot1024\cdot1024$

For 2147483648 steps, pi = 3.14159265358983 in 1.07845270074904 secs

---

#### Parallel task

MIN_BLK = $1024\cdot1024\cdot256$

For 2147483648 steps, pi = 3.14159265358983 in 1.61850937642157 secs

---

MIN_BLK = $1024\cdot1024\cdot64$

For 2147483648 steps, pi = 3.14159265358979 in 1.62034159712493 secs

---

MIN_BLK = $1024\cdot1024\cdot1024$

For 2147483648 steps, pi = 3.14159265358966 in 2.15922214649618 secs

---

### Manipulação de Efeitos Colaterais no Vetor

#### Vetor insert critical tempos obtidos:

Tempo com omp critical: 0.00138456 segundos.

Tempo com omp critical: 0.00101884 segundos.

Tempo com omp critical: 0.000911481 segundos.

#### Vetor insert pre alocado tempos obtidos:

Tempo com pré-alocação de memória: 0.000150934 segundos.

Tempo com pré-alocação de memória: 0.000150017 segundos.

Tempo com pré-alocação de memória: 0.000147307 segundos.


### 1. Qual abordagem teve melhor desempenho: `omp critical` ou pré-alocação de memória?
- **A abordagem com pré-alocação de memória** teve um desempenho significativamente melhor. Os tempos de execução foram em torno de `0.00015` segundos, enquanto a abordagem com `#pragma omp critical` variou de `0.00091` a `0.00138` segundos.

### 2. O uso de `omp critical` adicionou muito *overhead*? Como você pode justificar isso?
- Sim, o uso de `#pragma omp critical` adicionou um *overhead* significativo.
- A justificativa para esse *overhead* é que a região crítica impede que múltiplas threads acessem a operação `push_back` simultaneamente. Cada thread deve aguardar sua vez para adicionar elementos ao vetor, o que cria um gargalo de sincronização.
- Isso é especialmente problemático em situações onde a operação dentro da região crítica é simples e rápida, como no caso do `push_back`. O tempo gasto aguardando a exclusividade de acesso ao vetor pode ser maior do que o tempo necessário para realizar a operação.

### 3. A ordem dos dados no vetor foi mantida em ambas as abordagens?
- Sim, a ordem dos dados no vetor foi mantida em ambas as abordagens.
  - **Com `#pragma omp critical`**: Como cada `push_back` ocorre dentro de uma região crítica, as threads não podem interferir na ordem de inserção. Isso garante que os elementos sejam adicionados na ordem correta.
  - **Com pré-alocação de memória**: Cada thread escreve diretamente no índice correspondente (`vec[i] = conta_complexa(i)`), garantindo que os dados sejam preenchidos na ordem correta. Como cada índice é independente, não há condições de corrida nesse caso.

### Conclusão:
- A abordagem de pré-alocação de memória é mais eficiente em termos de tempo de execução e também simplifica o gerenciamento de paralelismo, já que elimina a necessidade de sincronização.
- `#pragma omp critical` deve ser evitado em casos onde a operação protegida é repetitiva e rápida, como `push_back` em um vetor, pois o *overhead* de sincronização pode anular os ganhos de paralelismo.
- A pré-alocação de memória também é uma prática recomendada para melhorar o desempenho em cenários paralelos, especialmente quando o tamanho do vetor é conhecido de antemão.


### Conclusão Geral

### Conclusão:

Com base nos testes realizados para diferentes abordagens de paralelismo, aqui estão as principais conclusões:

1. **Paralelização com `#pragma omp critical` vs. Pré-Alocação de Memória**:
   - A pré-alocação de memória demonstrou ser a abordagem mais eficiente em comparação com o uso de `#pragma omp critical` para evitar acessos simultâneos ao vetor.
   - O uso de `#pragma omp critical` adiciona um overhead significativo, pois limita a concorrência ao restringir o acesso ao vetor a uma única thread de cada vez, criando um gargalo.
   - A pré-alocação de memória com `vec.resize(N)` e o uso de um `#pragma omp parallel for` para preenchimento direto permitiu que todas as threads trabalhassem simultaneamente, eliminando a necessidade de sincronização e, consequentemente, melhorando o desempenho.

2. **Comparação de Schedulers (`static`, `dynamic`, `guided`, `auto`)**:
   - Nos testes com diferentes estratégias de agendamento (`scheduling`), `schedule(static)` foi a abordagem que apresentou o menor tempo médio em um cenário de carga leve, devido à divisão uniforme das tarefas e à baixa sobrecarga de gerenciamento.
   - `schedule(dynamic)` e `schedule(guided)` apresentaram tempos um pouco maiores, mas podem ser mais vantajosos em situações onde a carga de trabalho entre as threads não é uniforme.
   - A abordagem `schedule(auto)` mostrou-se competitiva em termos de desempenho, especialmente quando a carga de trabalho é equilibrada e previsível, mas depende das otimizações do compilador.

3. **Cálculo Paralelo do Pi**:
   - Na implementação recursiva para o cálculo de π, observou-se que a abordagem usando `#pragma omp parallel for` com redução foi mais direta e fácil de implementar em comparação com o uso de `#pragma omp task`.
   - A abordagem com `#pragma omp task` pode ser útil em problemas recursivos mais complexos ou onde a divisão da carga de trabalho é mais irregular, mas introduz mais complexidade de gerenciamento.
   - A pequena variação nas últimas casas decimais de π nas diferentes execuções foi explicada pelas diferenças na ordem de operações em ambientes paralelos, o que é esperado quando se trabalha com somas acumulativas em ponto flutuante.

### Abordagem Mais Eficiente para Problemas Recursivos e com Efeitos Colaterais:

- **Para Problemas Recursivos**:
   - Em problemas onde é necessário dividir o trabalho em subtarefas, a abordagem usando `#pragma omp task` pode ser mais adequada, especialmente se a carga de trabalho variar significativamente entre as tarefas.
   - No entanto, quando a carga de trabalho é previsível e uniforme, um simples `#pragma omp parallel for` com estratégias como `schedule(static)` pode ser mais eficiente e fácil de implementar.

- **Para Efeitos Colaterais (como modificação de vetores)**:
   - A pré-alocação de memória é geralmente a abordagem mais eficiente quando o tamanho do vetor é conhecido, pois elimina a necessidade de sincronização e permite que cada thread trabalhe de forma independente.
   - `#pragma omp critical` deve ser usado com cautela, apenas quando não é possível evitar a concorrência direta, pois pode introduzir um gargalo significativo em operações frequentes.

### Resultados Inesperados e Explicações:

- **Overhead de `#pragma omp critical`**:
   - Embora esperado um certo overhead, a diferença significativa entre `#pragma omp critical` e a abordagem de pré-alocação de memória foi notável nos testes. Isso destaca a importância de evitar seções críticas em operações que são rápidas e repetitivas.
   - Esse resultado pode ser explicado pela sobrecarga de sincronização e pelo tempo que as threads passam esperando pela exclusividade de acesso à seção crítica.

- **Variações nos Resultados do Cálculo de π**:
   - Pequenas diferenças nas casas decimais de π em cálculos paralelos foram inicialmente inesperadas, mas foram justificadas pela natureza dos cálculos de ponto flutuante em operações paralelas. A ordem das somas pode influenciar no resultado final devido aos erros de arredondamento, o que é natural em contextos de alta precisão.

### Conclusão Final:
- Em geral, a **pré-alocação de memória** e o uso de **estratégias de agendamento como `static`** se mostraram mais eficientes para problemas de paralelismo envolvendo vetores e somas.
- A escolha entre `#pragma omp task` e `#pragma omp parallel for` depende da complexidade do problema e do balanceamento da carga de trabalho. Para a maioria dos problemas recursivos simples, `parallel for` é mais fácil e eficiente.
- A escolha de técnicas deve ser baseada nas características específicas do problema, sempre levando em conta o balanceamento da carga e os efeitos colaterais nas estruturas de dados compartilhadas.